## Random Forest
- Random Forest model is a universall Model

In [18]:
import pandas as pd
df = pd.read_csv("diabetes.csv")
print(df.shape)
df.tail()

(768, 9)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1
767,1,93,70,31,0,30.4,0.315,23,0


In [2]:
# it is for checking the null values in the dataframe
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [3]:
x = df.drop('Outcome',axis = 1)
y = df['Outcome']

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.33,random_state = 3)

In [5]:
rfc = RandomForestClassifier()
rfc.fit(x_train,y_train)
rfc_predict = rfc.predict(x_test)

C:\Users\IIIT-rgukt\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [6]:
rfc.predict([[8,183,64,0,0,23.3,0.672,32]])

array([1], dtype=int64)

In [7]:
from sklearn.metrics import roc_auc_score

# Calculate roc auc
roc_value = roc_auc_score(y_test, rfc_predict)
roc_value

0.7067185109637939

In [8]:
# Evaluating Performances
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report,confusion_matrix

In [9]:
rfc_cv_score = cross_val_score(rfc,x,y,cv = 10,scoring = 'roc_auc')
rfc_cv_score

array([0.76777778, 0.75851852, 0.80814815, 0.69962963, 0.74185185,
       0.79740741, 0.83740741, 0.87074074, 0.81461538, 0.79384615])

In [10]:
print("==== Confusion Matrix ====")
print(confusion_matrix(y_test,rfc_predict))
print("==== Classification Report ====")
print(classification_report(y_test,rfc_predict))
print("==== All AUC Scores ====")
print(rfc_cv_score)
print()
print(" === Mean AUC Score - Random Forest ===: ",rfc_cv_score.mean())

==== Confusion Matrix ====
[[131  17]
 [ 50  56]]
==== Classification Report ====
              precision    recall  f1-score   support

           0       0.72      0.89      0.80       148
           1       0.77      0.53      0.63       106

    accuracy                           0.74       254
   macro avg       0.75      0.71      0.71       254
weighted avg       0.74      0.74      0.73       254

==== All AUC Scores ====
[0.76777778 0.75851852 0.80814815 0.69962963 0.74185185 0.79740741
 0.83740741 0.87074074 0.81461538 0.79384615]

 === Mean AUC Score - Random Forest ===:  0.788994301994302


In [11]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200,stop = 2000,num = 10)]
# number of features at every split
max_features = ['auto','sqrt']

# max depth
max_depth = [int(x) for x in np.linspace(100,500,num = 11)]
max_depth.append(None)

# create random grid
random_grid = {'n_estimators': n_estimators,'max_features':max_features,'max_depth':max_depth}

# random search of parameters
rfc_random = RandomizedSearchCV(estimator = rfc,param_distributions = random_grid,n_iter = 100,cv = 3,verbose = 2,random_state = 42,n_jobs = -1)

# fit the model
rfc_random.fit(x_train,y_train)

#print the results
print(rfc_random.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   17.7s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.2min finished


{'n_estimators': 1200, 'max_features': 'sqrt', 'max_depth': 420}


In [13]:
rfc = RandomForestClassifier(n_estimators = 1200,max_depth = 420,max_features = "sqrt")
rfc.fit(x_train,y_train)
rfc_predict = rfc.predict(x_test)
rfc_cv_score = cross_val_score(rfc,x,y,cv = 10,scoring = 'roc_auc')
print("==== Confusion Matrix ====")
print(confusion_matrix(y_test,rfc_predict))
print("==== Classification Report ====")
print(classification_report(y_test,rfc_predict))
print("==== All AUC Scores ====")
print(rfc_cv_score)
print()
print(" === Mean AUC Score - Random Forest ===: ",rfc_cv_score.mean())

==== Confusion Matrix ====
[[128  20]
 [ 40  66]]
==== Classification Report ====
              precision    recall  f1-score   support

           0       0.76      0.86      0.81       148
           1       0.77      0.62      0.69       106

    accuracy                           0.76       254
   macro avg       0.76      0.74      0.75       254
weighted avg       0.76      0.76      0.76       254

==== All AUC Scores ====
[0.78296296 0.8262963  0.83148148 0.74148148 0.81555556 0.86037037
 0.8637037  0.91296296 0.81230769 0.85846154]

 === Mean AUC Score - Random Forest ===:  0.8305584045584047


In [24]:
rfc.predict([[1,89,66,23,94,28.1,0.167,21]])

array([0], dtype=int64)